In [155]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import datetime

In [156]:
data = pd.read_csv('dataframe/champions.csv')

In [157]:
data['Temporada'] = pd.to_datetime(data['Temporada'])

# Ordenar el DataFrame por la columna 'Temporada'
data_sorted = data.sort_values(by='Temporada')

In [158]:
from sklearn.model_selection import train_test_split

# Selección de equipos de interés
equipo_de_interes = [1, 3, 5, 14, 16, 24, 28, 31]
equipo_data = data[data['Local'].isin(equipo_de_interes) | data['Visitante'].isin(equipo_de_interes)]

# Concatenación de goles locales y visitantes por temporada
serie_temporal_local = equipo_data.groupby(['Temporada', 'Local']).sum()['GolesLocal']
serie_temporal_visitante = equipo_data.groupby(['Temporada', 'Visitante']).sum()['GolesVisitante']

# Creación de la serie temporal para cada equipo
serie_temporal_por_equipo = pd.concat([serie_temporal_local, serie_temporal_visitante]).groupby(level=1).sum()

# División en conjunto de entrenamiento y prueba
train_data, test_data = train_test_split(serie_temporal_por_equipo, test_size=0.2, shuffle=False)



In [159]:
def arima_model(train, test, order):
    # Selección de la columna 'GolesLocal' de los datos de entrenamiento
    train_data = train['GolesLocal']
    
    # Crear el modelo ARIMA
    model = ARIMA(train_data, order=order)
    
    # Ajustar el modelo a los datos
    model_fit = model.fit()
    
    # Predecir los valores futuros
    forecast = model_fit.forecast(steps=len(test))
    
    return model_fit, forecast


In [161]:
import matplotlib.pyplot as plt

def series_plot(data, title):
    fig, axs = plt.subplots(len(data.columns), figsize=(10, 6*len(data.columns)), sharex=True)
    fig.suptitle(title)
    
    for i, equipo in enumerate(data.columns):
        equipo_data_local = equipo_data[equipo_data['Local'] == equipo]
        equipo_data_visitante = equipo_data[equipo_data['Visitante'] == equipo]
        
        axs[i].plot(equipo_data_local['Temporada'], equipo_data_local['GolesLocal'], marker='o', linestyle='-', label=f'{equipo} (Local)')
        axs[i].plot(equipo_data_visitante['Temporada'], equipo_data_visitante['GolesVisitante'], marker='x', linestyle='--', label=f'{equipo} (Visitante)')
        axs[i].set_ylabel('Goles Marcados')
        axs[i].legend()
        axs[i].grid(True)
    
    plt.xlabel('Temporada')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

serie_temporal_por_equipo_df = serie_temporal_por_equipo.unstack()

series_plot(serie_temporal_por_equipo_df, 'Serie Temporal de Goles Marcados por Equipo (Por Partido)')

ValueError: index must be a MultiIndex to unstack, <class 'pandas.core.indexes.base.Index'> was passed

In [ ]:
def evaluacion_modelo(test, forecast):
    errores = test - forecast

    # Métricas de evaluación
    mse = np.mean(errores ** 2)
    mae = np.mean(np.abs(errores))
    rmse = np.sqrt(mse)
    
    # Diccionario con las métricas
    metricas = {
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse
    }
    
    return metricas

In [ ]:
arima_model(train, test, order=(1, 1, 1))


/Users/carlotasanchezgonzalez/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/carlotasanchezgonzalez/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/carlotasanchezgonzalez/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/carlotasanchezgonzalez/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so w

(<statsmodels.tsa.arima.model.ARIMAResultsWrapper at 0x156a13450>,
 464    2.369016
 465    2.379070
 466    2.379344
 467    2.379352
 468    2.379352
          ...   
 576    2.379352
 577    2.379352
 578    2.379352
 579    2.379352
 580    2.379352
 Name: predicted_mean, Length: 117, dtype: float64)